In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import helper

import json
import os
import sys

In [2]:
course_schedule = []
NO_WINDOW = False
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440

WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

RETRY_INTERVAL = 60

In [3]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)

def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [5]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [6]:
temp = []

# ---------------

with open(r'./class_schedules/12 June - 18 June 2023.json', "r") as read_file:
    course_schedule = json.load(read_file)
# ---------------


Day_string = "14"


c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Wall', c)
# c = helper.search_courses_by_teacher('Char', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Basic', c)
c = helper.search_courses_by_teacher('Joe', c)
c = helper.search_courses_by_time('PM', c)
temp.extend(c)


courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 14 Jun Wed
-----------------------

Name: Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩
Teacher: Charmian
Time: 02:30 - 03:30 PM


Name: Aerial Yoga Basic 空中瑜珈入門
Teacher: Joe
Time: 12:45 - 01:45 PM


Name: Aerial Yoga 空中瑜珈(低空)
Teacher: Joe
Time: 08:45 - 09:45 PM




In [7]:
master_counter = 1

In [8]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [9]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [10]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [11]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [12]:
def reserve_class(driver, wait, course_to_be_booked):
    
    global master_counter
    global class_idx
    global num_classes

    stride =RETRY_INTERVAL / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    
#     reserve_class_log_dict = {}
    
#     reserve_class_log_dict['name'] = class_name
    driver.get(book_url)
    
    if click_book_this_class_now(driver, wait):
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
#         print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',FULL,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        if 0==0: #(master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
        
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=RETRY_INTERVAL)
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [13]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=30+(idx%len(course_to_be_booked))*10 )
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90]
[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


 
'1','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-12 13:45:58.101504'

'2','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-12 13:46:12.325390'

'3','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-12 13:46:26.497109'

'4','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-12 13:47:18.730699'

'5','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-12 13:47:32.935537'

'6','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-12 13:47:47.254620'

'7','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-12 13:48:39.362455'

'8','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-12 13:48:53.523035'

'9','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-12 13:49:07.839984'

'10','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-12 13:49:59.939693'

'11','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-12 13:50:14.143038'

'12','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-12 13:50:28.450959'

'13','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by '

'104','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-12 14:31:53.552415'

'105','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-12 14:32:07.326326'

'106','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-12 14:33:01.530250'

'107','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-12 14:33:14.148856'

'108','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-12 14:33:27.947620'

'109','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-12 14:34:22.142493'

'110','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-12 14:34:34.750086'

'111','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-12 14:34:48.575439'

'112','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-12 14:35:42.731069'

'113','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-12 14:35:55.378451'

'114','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-12 14:36:09.197237'

'115','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-12 14:37:03.340041'

'116','Aerial Yoga Basic 空中瑜

...

'1528','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 01:11:17.984192'

'1529','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 01:11:17.997132'

'1530','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 01:11:18.041956'

'1531','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 01:12:38.953366'

'1532','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 01:12:39.004505'

'1533','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 01:12:39.239385'

'1534','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 01:13:59.562954'

'1535','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 01:13:59.577195'

'1536','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 01:13:59.822211'

'1537','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 01:15:20.187633'

'1538','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 01:15:20.197637'

'1539','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 01:15:20.422978'

'1540','Iyengar 

'1628','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 01:55:40.573969'

'1629','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 01:55:40.599486'

'1630','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 01:57:00.896789'

'1631','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 01:57:01.214550'
'1631','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 01:57:01.214641'


'1633','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 01:58:21.522669'

'1634','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 01:58:21.808441'

'1635','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 01:58:21.825156'

'1636','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 01:59:42.140414'

'1637','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 01:59:42.438529'
'1637','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 01:59:42.438594'


'1639','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 02:01:02.725815'

'1640','Aerial Y

'1729','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 02:41:21.145438'

'1730','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 02:41:21.148573'

'1731','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 02:41:21.259450'

'1732','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 02:42:41.718036'

'1733','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 02:42:41.760663'

'1734','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 02:42:41.818848'

'1735','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 02:44:02.331871'

'1736','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 02:44:02.350423'

'1737','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 02:44:02.437111'

'1738','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 02:45:22.889764'

'1739','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 02:45:22.946435'

'1740','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 02:45:23.010067'

'1741','Aerial Y

'1831','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 03:27:01.192430'
'1831','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 03:27:01.192503'


'1833','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 03:27:01.257804'

'1834','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 03:28:21.789964'

'1835','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 03:28:21.808263'

'1836','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 03:28:21.845878'

'1837','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 03:29:42.357039'

'1838','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 03:29:42.395004'

'1839','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 03:29:42.462395'

'1840','Aerial Yoga 空中瑜珈(低空)' by 'Joe',FULL,'2023-06-13 03:31:02.949003'

'1841','Aerial Yoga Basic 空中瑜珈入門' by 'Joe',FULL,'2023-06-13 03:31:02.964434'

'1842','Iyengar Yoga (Wall Rope) - 艾揚格瑜珈 壁繩' by 'Charmian',FULL,'2023-06-13 03:31:03.081151'

'1843','Aerial Y

Job "reserve_class (trigger: date[2023-06-13 04:01:35 CST], next run at: 2023-06-13 04:01:35 CST)" raised an exception
Traceback (most recent call last):
  File "/Users/huohsien/anaconda2/envs/web/lib/python3.7/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-12-627058965afa>", line 16, in reserve_class
    driver.get(book_url)
  File "/Users/huohsien/anaconda2/envs/web/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "/Users/huohsien/anaconda2/envs/web/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/Users/huohsien/anaconda2/envs/web/lib/python3.7/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.Web